In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark= SparkSession \
 .builder \
 .master("local[*]") \
 .appName("test") \
 .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/28 01:34:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/28 01:34:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
spark

In [6]:
df_green=spark.read.parquet('../code/data/pq/green/*/*')

In [7]:
df_green.registerTempTable('green')

/home/murat/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [8]:
df_green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [9]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime>="2020-01-01 00:00:00"
GROUP BY
    1, 2
""")

In [10]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-17 10:00:00|  41| 638.2700000000001|            49|
|2020-01-02 09:00:00|  66|             229.4|            12|
|2020-01-16 20:00:00|  65|501.28000000000014|            32|
|2020-01-07 12:00:00|  66|             179.5|             9|
|2020-01-12 20:00:00| 247|36.900000000000006|             3|
|2020-01-12 15:00:00|  10|             75.92|             1|
|2020-01-14 20:00:00| 223|             85.86|            11|
|2020-01-22 10:00:00|  74| 1179.059999999999|            83|
|2020-01-13 18:00:00|  92|207.01999999999998|            11|
|2020-01-24 19:00:00| 130|308.59000000000003|            21|
|2020-01-23 20:00:00|  16|37.269999999999996|             2|
|2020-01-23 13:00:00| 181|281.76000000000005|            20|
|2020-01-29 09:00:00| 116|            302.97|            19|
|2020-01-06 21:00:00| 12

In [11]:
df_green_revenue \
 .repartition(20) \
 .write.parquet('../code/data/report/revenue/green',mode='overwrite')

In [13]:
df_yellow=spark.read.parquet('../code/data/pq/yellow/*/*')

In [14]:
df_yellow.registerTempTable('yellow')

In [15]:
df_yellow.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'airport_fee']

In [16]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime>="2020-01-01 00:00:00"
GROUP BY
    1, 2
""")

In [17]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-21 16:00:00| 151|1384.1999999999996|           102|
|2020-01-07 01:00:00| 234| 521.4000000000001|            33|
|2020-01-25 02:00:00| 170|1895.4199999999987|           109|
|2020-01-15 14:00:00|  13|2707.2699999999986|           103|
|2020-01-02 09:00:00| 161| 4299.929999999996|           242|
|2020-01-04 02:00:00|  79| 6893.030000000011|           426|
|2020-01-15 20:00:00| 239| 4298.739999999997|           298|
|2020-01-25 21:00:00| 230| 5935.560000000009|           388|
|2020-01-26 13:00:00| 236| 8182.920000000018|           569|
|2020-01-20 15:00:00| 113| 2593.589999999998|           165|
|2020-01-17 19:00:00| 170|  9634.39000000002|           555|
|2020-01-04 03:00:00| 158|1542.7899999999997|            92|
|2020-01-26 13:00:00| 239|6876.2300000000105|           475|
|2020-01-26 00:00:00| 26

In [18]:
df_yellow_revenue \
 .repartition(20) \
 .write.parquet('../code/data/report/revenue/yellow',mode='overwrite')

In [19]:
df_green_revenue=spark.read.parquet('../code/data/report/revenue/green')
df_yellow_revenue=spark.read.parquet('../code/data/report/revenue/yellow')

In [20]:
df_green_revenue_tmp= df_green_revenue \
 .withColumnRenamed('amount','green_amount') \
 .withColumnRenamed('number_records','green_number_records')

df_yellow_revenue_tmp= df_yellow_revenue \
 .withColumnRenamed('amount','yellow_amount') \
 .withColumnRenamed('number_records','yellow_number_records')

In [21]:
df_join=df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour','zone'],how='outer')

In [22]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   3|              null|                null|              25.0|                    1|
|2020-01-01 00:00:00|   4|              null|                null|1004.3000000000002|                   57|
|2020-01-01 00:00:00|   7| 769.7299999999997|                  45| 455.1700000000001|                   38|
|2020-01-01 00:00:00|  12|              null|                null|             107.0|                    6|
|2020-01-01 00:00:00|  37|            175.67|                   6|161.60999999999999|                    7|
|2020-01-01 00:00:00|  40|            168.98|                   8|             89.97|                    5|
|2020-01-01 00:00:00|  45|  

In [23]:
df_join.write.parquet('../code/data/report/total',mode='overwrite')

In [24]:
df_join=spark.read.parquet('../code/data/report/total')

In [25]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  14|              null|                null|               8.8|                    1|
|2020-01-01 00:00:00|  15|              null|                null|             34.09|                    1|
|2020-01-01 00:00:00|  17|195.03000000000003|                   9|220.20999999999998|                    8|
|2020-01-01 00:00:00|  25| 531.0000000000001|                  26|            324.35|                   16|
|2020-01-01 00:00:00|  32| 68.94999999999999|                   2|              18.0|                    1|
|2020-01-01 00:00:00|  43|107.52000000000001|                   6| 6539.510000000011|                  390|
|2020-01-01 00:00:00|  49|26

In [31]:
df_zones=spark.read.parquet('zones/')

In [32]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [33]:
df_result=df_join.join(df_zones,df_join.zone==df_zones.LocationID)

In [34]:
df_result.drop('LocationID','zone').write.parquet('tmp/revenue-zones')